## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool
import numpy as np
import lightgbm
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [2]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [3]:
optimizer = BayesianOptimization(simple_func, dict(a=(1, 3), b=(4, 7)))

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [4]:
optimizer.maximize(3, 2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        |  7.158    |  1.644    |  5.514    |
|  2        |  7.264    |  1.658    |  5.607    |
|  3        |  8.869    |  1.99     |  6.879    |
|  4        |  10.0     |  3.0      |  7.0      |
|  5        |  9.172    |  3.0      |  6.172    |


Great, now let's print the best parameters and the associated maximized target.

In [5]:
print(optimizer.max['params'])
optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [6]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [7]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [9]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [10]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [11]:
def label_enc(df_column):
    return LabelEncoder().fit_transform(df_column)

def get_harmonic(value, period):
    return 2*value* np.pi / period

def make_harmonic_features_sin(value, period=2400):
    return np.sin(get_harmonic(value, period))

def make_harmonic_features_cos(value, period=2400):
    return np.cos(get_harmonic(value, period))

def feature_eng(df):
    df['flight'] = df['Origin'] + df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [12]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [13]:
cat_columns = ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']
for column in cat_columns:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [14]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [15]:
categorical_features = ['Month',  'DayOfWeek'] + cat_columns

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [16]:
def lgb_eval(num_leaves, max_depth, lambda_l2, lambda_l1,
             min_child_samples, min_data_in_leaf):
    params = {
        'objective': 'binary',
        'metric': 'auc', 
        'is_unbalance': True,
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'lambda_l2': lambda_l2,
        'lambda_l1': lambda_l1,
        'num_threads': 20,
        'min_child_samples': int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        'learning_rate': 0.03,
        'subsample_freq': 5,
        'bagging_seed': 42,
        'verbosity': -1
    }

    lgtrain = lightgbm.Dataset(X_train, y_train, 
                               categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params, lgtrain, 1000, 
                            early_stopping_rounds=100,
                            stratified=True, nfold=3)

    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [17]:
params = {'num_leaves': (25, 4000),
          'max_depth': (5, 63),
          'lambda_l2': (0.0, 0.05),
          'lambda_l1': (0.0, 0.05),
          'min_child_samples': (50, 10000),
          'min_data_in_leaf': (100, 2000)}
lgbBO = BayesianOptimization(lgb_eval, params)

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7434   |  0.02417  |  0.04087  |  8.15     |  5.611e+0 |  1.544e+0 |  1.955e+0 |
|  2        |  0.723    |  0.03821  |  0.02678  |  47.67    |  3.641e+0 |  585.4    |  3.879e+0 |
|  3        |  0.717    |  0.04345  |  0.01419  |  8.565    |  7.296e+0 |  110.5    |  1.468e+0 |
|  4        |  0.7278   |  0.01323  |  0.02711  |  25.86    |  5.673e+0 |  914.0    |  2.727e+0 |
|  5        |  0.7435   |  0.03082  |  0.03131  |  18.47    |  983.6    |  1.948e+0 |  1.065e+0 |
|  6        |  0.7179   |  0.01775  |  0.001897 |  5.267    |  5.688e+0 |  104.9    |  1.653e+0 |
|  7        |  0.7431   |  0.02349  |  0.04751  |  43.95    |  7.367e+0 |  1.341e+0 |  2.557e+0 |
|  8        |  0.7336   |  0.006196 |  0.04782  |  48.03    |  4.3e+03  |  1.123e+0 |  1.414e+0 |
|  9        |  0.743

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [18]:
lgbBO.max

{'target': 0.7437792615384553,
 'params': {'lambda_l1': 0.042370071605676865,
  'lambda_l2': 0.02294226007600826,
  'max_depth': 46.59517208212181,
  'min_child_samples': 3238.264773177644,
  'min_data_in_leaf': 1468.1412257903564,
  'num_leaves': 2874.7171140751893}}

Review the process at each step by using the '.res[0]' function.

In [26]:
temp_df = pd.DataFrame([[i['target']] for i in lgbBO.res], columns=['target'])
params = [i['params'] for i in lgbBO.res]
pd.concat([temp_df, pd.DataFrame(params)], axis=1)

,target,lambda_l1,lambda_l2,max_depth,min_child_samples,min_data_in_leaf,num_leaves
0,0.743355,0.024172,0.040872,8.150071,5610.777317,1544.178002,1955.266215
1,0.722956,0.038209,0.026778,47.667379,3641.317797,585.409670,3878.752746
2,0.716951,0.043454,0.014190,8.565119,7296.376142,110.519680,1468.090209
3,0.727845,0.013228,0.027106,25.860395,5673.004414,913.991195,2726.767179
4,0.743482,0.030824,0.031309,18.474418,983.612104,1947.564366,1065.282370
5,0.717851,0.017751,0.001897,5.267476,5687.835152,104.944291,1652.546207
6,0.743127,0.023493,0.047511,43.951592,7367.218465,1341.354137,2556.566287
7,0.733596,0.006196,0.047823,48.025301,4299.724065,1122.827522,1414.123456
8,0.743386,0.008952,0.012615,15.468124,4357.893495,1799.187467,525.213043
9,0.717811,0.000015,0.003431,26.985520,980.544913,467.097995,3929.285370
